In [ ]:
!pip install ipython-sql
!pip install SQLAlchemy
!pip install sqlite3

In [9]:
import fetch_data
df = fetch_data.fromCSV()
df.head(3)

INFO:fetch_data:Fetching file by url on 2025-02-15 Time 20:02:37+00:00 UTC


,kjønn,utdanning,erfaring,arbeidssted,arbeidssituasjon,fag,lønn,bonus?
0,mann,4,7,Agder,"in-house, privat sektor",AI / maskinlæring,865000,Nei
1,mann,5,2,Nordland,"in-house, offentlig/kommunal sektor",AI / maskinlæring,756000,Nei
2,mann,4,30,Oslo,konsulent,AI / maskinlæring,1500000,Nei


In [10]:
df = df.rename(columns = {'kjønn':'sex',
                          'utdanning':'education',
                          'erfaring':'experience',
                          'arbeidssted':'workplace',
                          'arbeidssituasjon':'work_situation',
                          'fag':'subject',
                          'lønn':'salary'
                         })
df.head(3)

,sex,education,experience,workplace,work_situation,subject,salary,bonus?
0,mann,4,7,Agder,"in-house, privat sektor",AI / maskinlæring,865000,Nei
1,mann,5,2,Nordland,"in-house, offentlig/kommunal sektor",AI / maskinlæring,756000,Nei
2,mann,4,30,Oslo,konsulent,AI / maskinlæring,1500000,Nei


In [11]:
import sqlite3
data_base_name = 'hw_03_data_base.db'
con = sqlite3.connect(data_base_name)

In [12]:
table_name = 'data'
df.to_sql(table_name, con=con, if_exists='replace')

cur = con.cursor()
for row in cur.execute(f"SELECT * FROM {table_name} LIMIT 10"):
        print(row)

(0, 'mann', 4, 7, 'Agder', 'in-house, privat sektor', 'AI / maskinlæring', 865000, 'Nei')
(1, 'mann', 5, 2, 'Nordland', 'in-house, offentlig/kommunal sektor', 'AI / maskinlæring', 756000, 'Nei')
(2, 'mann', 4, 30, 'Oslo', 'konsulent', 'AI / maskinlæring', 1500000, 'Nei')
(3, 'mann', 5, 25, 'Oslo', 'konsulent', 'AI / maskinlæring', 1200000, 'Nei')
(4, 'mann', 9, 18, 'Oslo', 'in-house, privat sektor', 'AI / maskinlæring', 1200000, 'Nei')
(5, 'mann', 2, 14, 'Oslo', 'in-house, privat sektor', 'AI / maskinlæring', 839000, 'Nei')
(6, 'kvinne', 5, 9, 'Oslo', 'konsulent', 'AI / maskinlæring', 1200000, 'Ja')
(7, 'mann', 6, 9, 'Oslo', 'in-house, privat sektor', 'AI / maskinlæring', 980000, 'Ja')
(8, 'annet / ønsker ikke oppgi', 5, 5, 'Oslo', 'in-house, privat sektor', 'AI / maskinlæring', 750000, 'Nei')
(9, 'mann', 5, 5, 'Oslo', 'konsulent', 'AI / maskinlæring', 910000, 'Nei')


In [13]:
con.close()